In [1]:
import json
from shapely.geometry import LineString
import pandas as pd
import geopandas as gpd

In [2]:
route_df = pd.read_json(r'C:\Users\capov\smarts-sandbox\generator\output.json')
route_gdf = gpd.GeoDataFrame(route_df, geometry=gpd.points_from_xy(route_df.longitude, route_df.latitude))
route_gdf = route_gdf.loc[:, ['timestamp', 'azimuth', 'speed', 'geometry']]
# pd.options.display.max_rows = None
display(route_gdf)

,timestamp,azimuth,speed,geometry
0,2022-06-21 14:33:26+00:00,47.536787,3.6,POINT (50.24004 53.26139)
1,2022-06-21 14:33:27+00:00,47.538001,7.2,POINT (50.24005 53.26139)
2,2022-06-21 14:33:27+00:00,47.540005,10.8,POINT (50.24007 53.26141)
3,2022-06-21 14:33:28+00:00,47.542805,14.4,POINT (50.24010 53.26142)
4,2022-06-21 14:33:28+00:00,47.546413,18.0,POINT (50.24013 53.26144)
...,...,...,...,...
210,2022-06-21 14:35:11+00:00,136.072091,20.0,POINT (50.24370 53.26621)
211,2022-06-21 14:35:11+00:00,136.226923,20.0,POINT (50.24373 53.26619)
212,2022-06-21 14:35:12+00:00,136.443502,20.0,POINT (50.24376 53.26617)
213,2022-06-21 14:35:12+00:00,136.803794,20.0,POINT (50.24379 53.26615)


In [3]:
edges = []
with open('road_graph.json') as road_graph_file:
    road_graph = json.load(road_graph_file)
    
for node in road_graph['nodes']:
    for neighbour in node['neighbours']:
        edges.append({
            'start_node_id': node['node_id'],
            'end_node_id': neighbour['id'],
            'azimuth': neighbour['azimuth'],
            'speed_limit': neighbour['speed_limit'],
            'min_speed_limit': neighbour['min_speed_limit']
        })
edges_df = pd.DataFrame(edges)
        
road_graph_df = pd.DataFrame({
    'node_id': [node['node_id'] for node in road_graph['nodes']],
    'latitude': [node['latitude'] for node in road_graph['nodes']],
    'longitude': [node['longitude'] for node in road_graph['nodes']],
})

display(road_graph_df)

,node_id,latitude,longitude
0,0,53.261388,50.240043
1,1,53.262072,50.241290
2,2,53.261903,50.241567
3,3,53.264023,50.244857
4,4,53.267386,50.251118
5,5,53.265452,50.242612
6,6,53.266362,50.241183
7,7,53.265436,50.239416
8,8,53.266875,50.240394
9,9,53.268104,50.238486


In [4]:
edges_df = pd.merge(edges_df, road_graph_df, left_on='start_node_id', right_on='node_id')
edges_df = edges_df.loc[:, ['start_node_id', 'latitude', 'longitude', 'end_node_id', 'azimuth', 'speed_limit', 'min_speed_limit']]
edges_df.rename(columns={'latitude': 'start_lat', 'longitude': 'start_lon'}, inplace = True) 
edges_df = pd.merge(edges_df, road_graph_df, left_on='end_node_id', right_on='node_id')
edges_df = edges_df.loc[:, ['start_node_id', 'end_node_id', 'start_lat', 'start_lon', \
                            'latitude', 'longitude', 'azimuth', 'speed_limit', 'min_speed_limit']]
edges_df.rename(columns={'latitude': 'end_lat', 'longitude': 'end_lon'}, inplace = True)
edges_df = edges_df.assign(geometry=edges_df.apply( \
    lambda x: LineString([(x['start_lon'], x['start_lat']), (x['end_lon'], x['end_lat'])]), axis=1))
edges_df = edges_df.loc[:, ['start_node_id', 'end_node_id', 'geometry', 'azimuth', 'speed_limit', 'min_speed_limit']]

edges_gdf = gpd.GeoDataFrame(edges_df, geometry=edges_df.geometry)
display(edges_gdf)

C:\Users\capov\AppData\Roaming\jupyterlab-desktop\jlab_server\lib\site-packages\pandas\core\dtypes\cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


,start_node_id,end_node_id,geometry,azimuth,speed_limit,min_speed_limit
0,0,1,"LINESTRING (50.24004 53.26139, 50.24129 53.26207)",47.478510,60,40
1,2,1,"LINESTRING (50.24157 53.26190, 50.24129 53.26207)",315.566824,60,40
2,1,2,"LINESTRING (50.24129 53.26207, 50.24157 53.26190)",135.566602,60,40
3,1,3,"LINESTRING (50.24129 53.26207, 50.24486 53.26402)",47.579146,60,40
4,4,3,"LINESTRING (50.25112 53.26739, 50.24486 53.26402)",228.051502,60,40
5,3,4,"LINESTRING (50.24486 53.26402, 50.25112 53.26739)",48.046346,60,40
6,3,5,"LINESTRING (50.24486 53.26402, 50.24261 53.26545)",320.901531,60,40
7,5,6,"LINESTRING (50.24261 53.26545, 50.24118 53.26636)",316.795854,60,40
8,7,6,"LINESTRING (50.23942 53.26544, 50.24118 53.26636)",48.774726,60,40
9,6,7,"LINESTRING (50.24118 53.26636, 50.23942 53.26544)",228.776142,60,40


In [11]:
joined_gdf = gpd.sjoin_nearest(route_gdf, edges_gdf, max_distance=5, distance_col='distance')

display(joined_gdf)

C:\Users\capov\AppData\Local\Temp\ipykernel_8560\2696745280.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: +init=epsg:4326 +type=crs

  joined_gdf = gpd.sjoin_nearest(route_gdf, edges_gdf, max_distance=5, distance_col='distance')
C:\Users\capov\AppData\Roaming\jupyterlab-desktop\jlab_server\lib\site-packages\geopandas\array.py:340: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


,timestamp,azimuth_left,speed,geometry,index_right,start_node_id,end_node_id,azimuth_right,speed_limit,min_speed_limit,distance
0,2022-06-21 14:33:26+00:00,47.536787,3.6,POINT (50.24004 53.26139),0,0,1,47.478510,60,40,3.130495e-06
1,2022-06-21 14:33:27+00:00,47.538001,7.2,POINT (50.24005 53.26139),0,0,1,47.478510,60,40,2.815912e-07
2,2022-06-21 14:33:27+00:00,47.540005,10.8,POINT (50.24007 53.26141),0,0,1,47.478510,60,40,4.293123e-07
3,2022-06-21 14:33:28+00:00,47.542805,14.4,POINT (50.24010 53.26142),0,0,1,47.478510,60,40,6.980368e-07
4,2022-06-21 14:33:28+00:00,47.546413,18.0,POINT (50.24013 53.26144),0,0,1,47.478510,60,40,1.039673e-06
...,...,...,...,...,...,...,...,...,...,...,...
210,2022-06-21 14:35:11+00:00,136.072091,20.0,POINT (50.24370 53.26621),14,10,11,134.300831,20,0,2.954353e-05
211,2022-06-21 14:35:11+00:00,136.226923,20.0,POINT (50.24373 53.26619),14,10,11,134.300831,20,0,6.365289e-05
212,2022-06-21 14:35:12+00:00,136.443502,20.0,POINT (50.24376 53.26617),14,10,11,134.300831,20,0,9.773208e-05
213,2022-06-21 14:35:12+00:00,136.803794,20.0,POINT (50.24379 53.26615),14,10,11,134.300831,20,0,1.316943e-04


In [6]:
def define_over_speed_class(over_speed):
    if over_speed < 0:
        return 'under_min_speed_limit'
    elif 0 < over_speed <= 20:
        return '+(0-20]'
    elif 20 < over_speed <= 40:
        return '+(20-40]'
    elif 40 < over_speed <= 60:
        return '+(40-60]'
    elif 60 < over_speed <= 80:
        return '+[60-80]'
    elif over_speed > 80:
        return '+[80+]'

def define_over_or_under_speed(row):
    if row['speed'] > row['speed_limit']:
        return row['speed'] - row['speed_limit']
    elif row['speed'] < row['min_speed_limit']:
        return row['speed'] - row['min_speed_limit']
    else:
        return None
    
joined_gdf = joined_gdf \
                 .assign(over_speed=joined_gdf \
                 .apply(lambda x: define_over_or_under_speed(x), axis=1))
joined_gdf = joined_gdf.assign(over_speed_class=joined_gdf.apply(lambda x: define_over_speed_class(x['over_speed']), axis=1))
events_for_visualisation = joined_gdf.query("over_speed_class.isnull() != True")
joined_gdf = joined_gdf.where(pd.notnull(joined_gdf), None)

pd.options.display.max_rows = 10
display(joined_gdf)
joined_gdf['timestamp'] = joined_gdf['timestamp'].astype(str)
joined_gdf['geometry'] = joined_gdf['geometry'].astype(str)
event = []
for i, row in joined_gdf.iterrows():
    if row['over_speed_class'] != joined_gdf.iloc[i-1]['over_speed_class'] and row['over_speed_class'] != None:
        event.append(({'point': row['geometry'], 'over_speed_class': row['over_speed_class']}, joined_gdf \
                     .loc[:, ['timestamp', 'azimuth_left', 'speed', 'speed_limit', 'min_speed_limit', 'geometry', 'over_speed_class']] \
                     .rename(columns={'azimuth_left': 'azimuth'}) \
                     .iloc[i-5:i+1] \
                     .to_dict(orient="records")))

with open('events.json', 'w') as outfile:
    json.dump(event, outfile)


,timestamp,azimuth_left,speed,geometry,index_right,start_node_id,end_node_id,azimuth_right,speed_limit,min_speed_limit,over_speed,over_speed_class
0,2022-06-21 14:33:26+00:00,47.536787,3.6,POINT (50.24004 53.26139),0,0,1,47.478510,60,40,-36.4,under_min_speed_limit
1,2022-06-21 14:33:27+00:00,47.538001,7.2,POINT (50.24005 53.26139),0,0,1,47.478510,60,40,-32.8,under_min_speed_limit
2,2022-06-21 14:33:27+00:00,47.540005,10.8,POINT (50.24007 53.26141),0,0,1,47.478510,60,40,-29.2,under_min_speed_limit
3,2022-06-21 14:33:28+00:00,47.542805,14.4,POINT (50.24010 53.26142),0,0,1,47.478510,60,40,-25.6,under_min_speed_limit
4,2022-06-21 14:33:28+00:00,47.546413,18.0,POINT (50.24013 53.26144),0,0,1,47.478510,60,40,-22.0,under_min_speed_limit
...,...,...,...,...,...,...,...,...,...,...,...,...
210,2022-06-21 14:35:11+00:00,136.072091,20.0,POINT (50.24370 53.26621),14,10,11,134.300831,20,0,NaN,None
211,2022-06-21 14:35:11+00:00,136.226923,20.0,POINT (50.24373 53.26619),14,10,11,134.300831,20,0,NaN,None
212,2022-06-21 14:35:12+00:00,136.443502,20.0,POINT (50.24376 53.26617),14,10,11,134.300831,20,0,NaN,None
213,2022-06-21 14:35:12+00:00,136.803794,20.0,POINT (50.24379 53.26615),14,10,11,134.300831,20,0,NaN,None


C:\Users\capov\AppData\Roaming\jupyterlab-desktop\jlab_server\lib\site-packages\geopandas\geodataframe.py:1455: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [7]:
import folium
import json

my_map = folium.Map(location=[53.261382, 50.240032],
                tiles='openstreetmap', # оно такое по умолчанию
                zoom_start=13)

with open('road_graph.json') as road_graph:
    road_graph_data = json.load(road_graph)

with open(r'C:\Users\capov\smarts-sandbox\generator\output.json') as route:
    route_data = json.load(route)

edges_gdf.crs = {'init' :'epsg:4326'}
folium.Choropleth(
    edges_gdf[edges_gdf.geometry.length>0.00001],
    line_weight=3,
    line_color='blue'
).add_to(my_map)

for point in route_data:
    folium.Marker(
        location=[point['latitude'], point['longitude']],
        tooltip='Метка').add_to(my_map)
    
for i, point in events_for_visualisation.iterrows():
    folium.Marker(
        location=[point['geometry'].y, point['geometry'].x],
        icon=folium.Icon(color="red" if '+' in point['over_speed_class'] else "beige")).add_to(my_map)
    
for node in road_graph_data['nodes']:
    folium.Marker(
        location=[node['latitude'], node['longitude']],
        tooltip=node['node_id'],
        icon=folium.Icon(color="green")).add_to(my_map)

my_map

C:\Users\capov\AppData\Roaming\jupyterlab-desktop\jlab_server\lib\site-packages\pyproj\crs\crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\capov\AppData\Local\Temp\ipykernel_8560\3127379527.py:16: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  edges_gdf[edges_gdf.geometry.length>0.00001],


In [9]:
!python speeding_events.py C:\Users\capov\smarts-sandbox\generator\output.json road_graph.json events.json

C:\Users\capov\AppData\Roaming\jupyterlab-desktop\jlab_server\lib\site-packages\pandas\core\dtypes\cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
C:\Users\capov\AppData\Roaming\jupyterlab-desktop\jlab_server\lib\site-packages\geopandas\geodataframe.py:1455: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")
